In [1]:
import re
import nltk
import pandas as pd

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHAZZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df=pd.read_csv('spam.csv', sep='\t', names=['label', 'message'])

In [3]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [5]:
ps=PorterStemmer()
corpus=[]

In [6]:
for i in range(0, len(df)):
    msg=re.sub('[^a-zA-Z]', ' ', df['message'][i])
    msg=msg.lower()
    msg=msg.split()
    
    msg=[ps.stem(word) for word in msg if not word in stopwords.words('english')]
    msg=' '.join(msg)
    corpus.append(msg)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv=CountVectorizer(max_features=2500)

In [9]:
X=cv.fit_transform(corpus).toarray()

In [10]:
y=pd.get_dummies(df['label'])

In [11]:
y=y.iloc[:, 1].values

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [17]:
from sklearn.naive_bayes import MultinomialNB

In [18]:
spam_detect_model=MultinomialNB().fit(X_train, y_train)

In [19]:
y_pred=spam_detect_model.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
confusion_matrix(y_test, y_pred)

array([[949,  13],
       [  9, 144]], dtype=int64)

In [23]:
spam_detect_model.score(X_test, y_test)

0.9802690582959641

In [37]:
def is_spam(msg):
    msg=re.sub('[^a-zA-Z]', ' ', msg)
    msg=msg.lower()
    msg=msg.split()
    
    msg=[ps.stem(word) for word in msg if not word in stopwords.words('english')]
    msg=' '.join(msg)
    
    temp=cv.transform([msg]).toarray()
    return spam_detect_model.predict(temp)

In [55]:
is_spam("click yourself")

array([0], dtype=uint8)